In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, SimpleRNN, LSTM, Embedding
from tensorflow.keras.utils import to_categorical

# Load the dataset
data = pd.read_csv("healthcare_dataset.csv")

# Data preprocessing
# Encode categorical columns
label_encoders = {}
for column in ["Gender", "Smoking_Status"]:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Separate features and target
target = "Readmission_Risk"
X = data.drop(columns=[target])
y = data[target]

# Standardize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert target to categorical
num_classes = len(y.unique())
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Define models

def build_ffnn(input_dim, output_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dense(32, activation='relu'),
        Dense(output_dim, activation='softmax')
    ])
    return model

def build_cnn(input_shape, output_dim):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(output_dim, activation='softmax')
    ])
    return model

def build_rnn(input_shape, output_dim):
    model = Sequential([
        SimpleRNN(32, activation='relu', input_shape=input_shape),
        Dense(output_dim, activation='softmax')
    ])
    return model

def build_lstm(input_shape, output_dim):
    model = Sequential([
        LSTM(32, activation='relu', input_shape=input_shape),
        Dense(output_dim, activation='softmax')
    ])
    return model

# Train and evaluate models
def train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=10, batch_size=32):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    return history, test_loss, test_accuracy

# Reshape for CNN, RNN, and LSTM
input_shape_cnn = (X_train.shape[1], 1)
X_train_cnn = X_train.reshape(-1, X_train.shape[1], 1)
X_test_cnn = X_test.reshape(-1, X_test.shape[1], 1)

input_shape_rnn = (X_train.shape[1], 1)
X_train_rnn = X_train_cnn  # Same reshaping as CNN
X_test_rnn = X_test_cnn

input_shape_lstm = (X_train.shape[1], 1)
X_train_lstm = X_train_cnn  # Same reshaping as CNN
X_test_lstm = X_test_cnn

# FFNN
ffnn_model = build_ffnn(X_train.shape[1], num_classes)
ffnn_history, ffnn_loss, ffnn_accuracy = train_and_evaluate(ffnn_model, X_train, y_train, X_test, y_test)

# CNN
cnn_model = build_cnn(input_shape_cnn, num_classes)
cnn_history, cnn_loss, cnn_accuracy = train_and_evaluate(cnn_model, X_train_cnn, y_train, X_test_cnn, y_test)

# RNN
rnn_model = build_rnn(input_shape_rnn, num_classes)
rnn_history, rnn_loss, rnn_accuracy = train_and_evaluate(rnn_model, X_train_rnn, y_train, X_test_rnn, y_test)

# LSTM
lstm_model = build_lstm(input_shape_lstm, num_classes)
lstm_history, lstm_loss, lstm_accuracy = train_and_evaluate(lstm_model, X_train_lstm, y_train, X_test_lstm, y_test)

# Plot performance metrics
def plot_history(histories, labels):
    plt.figure(figsize=(12, 6))
    for history, label in zip(histories, labels):
        plt.plot(history.history['accuracy'], label=f'{label} Train')
        plt.plot(history.history['val_accuracy'], label=f'{label} Val')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

plot_history([ffnn_history, cnn_history, rnn_history, lstm_history], ['FFNN', 'CNN', 'RNN', 'LSTM'])

# Compare results
results = pd.DataFrame({
    'Model': ['FFNN', 'CNN', 'RNN', 'LSTM'],
    'Test Accuracy': [ffnn_accuracy, cnn_accuracy, rnn_accuracy, lstm_accuracy],
    'Test Loss': [ffnn_loss, cnn_loss, rnn_loss, lstm_loss]
})

print(results)

# Identify best-performing model
best_model = results.loc[results['Test Accuracy'].idxmax()]
print("Best Performing Model:", best_model)
